# CUDA编程模型---初识CUDA

### 本次实验将介绍如何：
1. 编写第一个Cuda程序
2. 利用NVCC进行编译
3. 编写Makefile文件
4. 利用nvprof查看程序性能

----
## 1.编写第一个Cuda程序
- 关键词："\_\_global\_\_" ,  <<<...>>>  ,  .cu


在当前的目录下创建一个名为hello_cuda.cu的文件，编写第一个Cuda程序：
- 当我们编写一个hello_word程序的时候，我们通常会这样写：

    #include <stdio.h>

    void hello_from_cpu()
    {
        printf("Hello World from the CPU!\n");
    }

    int main(void)
    {
        hello_from_cpu();
        return 0;
    }

- 如果我们要把它改成调用GPU的时候，我们需要在void hello_from_cpu()之前加入 \_\_global\_\_标识符，并且在调用这个函数的时候添加<<<...>>>来设定执行设置
- 在当前的目录下创建一个名为[hello_cuda.cu](hello_cuda.cu)的文件，更改上述程序，将它改为在GPU上执行的程序，如果遇到麻烦，请参考[hello_cuda02.cu](hello_cuda02.cu)

----
## 2.编写完成之后，我们要开始编译并执行程序，在这里我们可以利用nvcc进行编译，指令如下：

In [3]:
!/usr/local/cuda/bin/nvcc -arch=compute_80 -code=sm_80 hello_cuda.cu -o hello_cuda -run

Hello World from the GPU!


----
## 3.这里我们也可以利用编写Makefile的方式来进行编译，一个简单的例子可以参考[Makefile](Makefile)

In [10]:
!make

/usr/local/cuda/bin/nvcc --device-c hello_from_gpu.cu -o hello_from_gpu.o
/usr/local/cuda/bin/nvcc  hello_cuda02-test.cu hello_from_gpu.o -o ./hello_cuda_multi_file


然后我们就可以得到一个名为hello_cuda.exe的程序，我们开始执行一下

In [11]:
!./hello_cuda

Hello World from the GPU!


接下来我们尝试多个文件协同编译, 修改[Makefile](Makefile)文件:
1. 编译hello_from_gpu.cu文件生成hello_from_gpu.o
2. 编译hello_cuda02-test.cu和上一步生成的hello_from_gpu.o, 生成./hello_cuda_multi_file

如果遇到麻烦, 请参考[Makefile_Multi_file](Makefile_Multi_file)

In [12]:
!make

make: 'hello_cuda_multi_file' is up to date.


In [13]:
!./hello_cuda_multi_file

In [14]:
!make clean

rm -rf ./hello_cuda_multi_file
rm -rf *.o


这时，您已经完成了第一个Cuda程序，接下来修改<<<...>>>里面的信息，查看显示效果，如果遇到麻烦，请参考[hello_cuda03.cu](hello_cuda03.cu)

----
## 4.利用nvprof进行查看程序性能

In [15]:
!/usr/local/cuda/bin/nsys nvprof --print-api-trace ./hello_cuda

The --print-api-trace  switch is ignored by nsys.

Hello World from the GPU!
Generating '/tmp/nsys-report-43b5.qdstrm'
[1/7] [========================100%] report2.nsys-rep
[2/7] [========================100%] report2.sqlite
[3/7] Executing 'nvtxsum' stats report
SKIPPED: /mnt/CUDA_on_ARM/01_1.4/report2.sqlite does not contain NV Tools Extension (NVTX) data.
[4/7] Executing 'cudaapisum' stats report

CUDA API Statistics:

 Time (%)  Total Time (ns)  Num Calls    Avg (ns)      Med (ns)     Min (ns)    Max (ns)   StdDev (ns)          Name         
 --------  ---------------  ---------  ------------  ------------  ----------  ----------  -----------  ---------------------
    100.0       1034527581          1  1034527581.0  1034527581.0  1034527581  1034527581          0.0  cudaLaunchKernel     
      0.0            47468          1       47468.0       47468.0       47468       47468          0.0  cudaDeviceSynchronize

[5/7] Executing 'gpukernsum' stats report

CUDA Kernel Statistics:

 

In [16]:
!/usr/local/cuda/bin/nsys --print-api-trace ./hello_cuda

Unknown command: --print-api-trace
 usage: nsys [--version] [--help] <command> [<args>] [application] [<application args>]

 The most commonly used nsys commands are:
	profile       Run an application and capture its profile into a QDSTRM file.
	launch        Launch an application ready to be profiled.
	start         Start a profiling session.
	stop          Stop a profiling session and capture its profile into a QDSTRM file.
	cancel        Cancel a profiling session and discard any collected data.
	stats         Generate statistics from an existing nsys-rep or SQLite file.
	status        Provide current status of CLI or the collection environment.
	shutdown      Disconnect launched processes from the profiler and shutdown the profiler.
	sessions list List active sessions.
	export        Export nsys-rep file into another format.
	analyze       Run rules on an existing nsys-rep or SQLITE file.
	nvprof        Translate nvprof switches to nsys switches and execute collection.

 Use 'nsys 

- Profiling result：是GPU（kernel函数）上运行的时间
- API calls：是在cpu上测量的程序调用API的时间

课后作业：
1. 利用Makefile规则，尝试编写批量编译工具，比如：同时编译5个cuda程序。
2. 利用Makefile规则，尝试加入链接库，比如：加入cuBLAS库编译cuda程序。
3. 阅读Cuda sample code，尝试编写程序得到当前GPU的属性参数等。
4. 阅读[nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html#nvprof-overview) 说明文档，了解更多nvprof的使用方法，为后续课程中使用做